In [15]:
import pandas as pd
from ipywidgets import  widgets, interact, Dropdown, Layout, HTML, AppLayout, VBox, Label, HBox, Output
#import plotly.graph_objects as go
import pickle
import gensim.corpora as corpora
import nltk
import spacy #used for lemmatization
nltk.download('stopwords')
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
import pyLDAvis.gensim_models
#from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.utils import simple_preprocess
import json
!python -m spacy download en_core_web_sm
#!pip install plotly
#Must run this if this notebook is executed in a JupyterLab >3.0 .
# conda install -n base -c conda-forge jupyterlab_widgets

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 12.8 MB 24.5 MB/s            
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# Create text widget for output
output_text = widgets.Text()

# Create text widget for input
input_text = widgets.Text()

# Define function to bind value of the input to the output variable 
def bind_input_to_output(sender):
    output_text.value = input_text.value
input_text.on_submit(bind_input_to_output)

In [4]:
#Load tfidf model
lda_model_tfidf_lem_5 = 'lda_model_tfidf_lem_5.pkl'
with open(lda_model_tfidf_lem_5, 'rb') as f:
    lda_model_tfidf_lem_5 = pickle.load(f)  

In [5]:
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[x*\/]', '', text) #remove those dates with the format of XX/XX OR XXXX/XX/XX
    text = re.sub('x', '', text)#remove \n
    text = re.sub('\n', '', text)#remove \n
    return text

stop_words = stopwords.words('english')
stop_words.extend(['sent','told','received','said','asked','made','contacted','know','spoke','company','will','year','month'])

# def sent_to_words(sentences):
#     for sentence in sentences:
#         yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15,threshold=100):
    bigram = gensim.models.Phrases(words, min_count = bi_min,threshold=threshold)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

def trigrams(words, tri_min=15,threshold=100):
    bigram = gensim.models.Phrases(words, min_count=tri_min, threshold=threshold)
    trigram = gensim.models.Phrases(bigram[words],threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [6]:
spacy.prefer_gpu() # or spacy.require_gpu()
nlp = spacy.load("en_core_web_sm", disable=['ner', 'parser'])
def run_model(text_input):
    clean_txt=clean_text(text_input)
    words=gensim.utils.simple_preprocess(str(clean_txt), deacc=True)
    data_words_nostops=[[word for word in words if word not in stop_words]]
    bigram_mod = bigrams(data_words_nostops)
    bigrams_output = [bigram_mod[w] for w in data_words_nostops]

    data_lemmatized_bigrams = lemmatization(bigrams_output)
    id2word_dic_tfidf='lda_model_tfidf_lem_5.model.id2word'
    id2word_tfidf = corpora.Dictionary.load(id2word_dic_tfidf)

    texts=data_lemmatized_bigrams
# Term Document Frequency
    corpus = [id2word_tfidf.doc2bow(text) for text in texts]
    for i, row_list in enumerate(lda_model_tfidf_lem_5[corpus]):
        row = row_list[0] if lda_model_tfidf_lem_5.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
    # print(row)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model_tfidf_lem_5.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                result=int(topic_num)
                result2=prop_topic, 
               
    out = Output(layout={'border': '1px solid white'})
    with out:
        display(HTML(value=f'Predicted Topic: {result},  Perc Contribution: {result2},  Keywords:{topic_keywords}'))
    return out
        



In [11]:
from gensim.models import TfidfModel
#load model 
lda_model_tfidf_lem_5 = 'lda_model_tfidf_lem_5.pkl'
with open(lda_model_tfidf_lem_5, 'rb') as f:
    lda_model_tfidf_lem_5 = pickle.load(f)  

    #load lemmatized bigrams    
with open('data_lemmatized_bigrams.txt', 'r') as f:
    data_lemmatized_bigrams = json.loads(f.read())
texts=data_lemmatized_bigrams

In [17]:
id2word_dic_tfidf='lda_model_tfidf_lem_5.model.id2word'
id2word_tfidf = corpora.Dictionary.load(id2word_dic_tfidf)


# Term Document Frequency
corpus = [id2word_tfidf.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [18]:
#Visualize the tfidf with top 10 topics
vis2 = pyLDAvis.gensim_models.prepare(lda_model_tfidf_lem_5, corpus_tfidf, id2word_tfidf)
pyLDAvis.display(vis2)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [8]:
interact(run_model,text_input=str(input_text.value));

interactive(children=(Text(value='', description='text_input'), Output()), _dom_classes=('widget-interact',))